# Traffic sign recognition
#### German traffic sign benchmark dataset

In [1]:
# Import main libraries
import pandas as pd
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display
%matplotlib inline

# Import resource
from src.preprocessors.HistogramEqualizer import HistogramEqualizer
from src.preprocessors.Normalizer import Normalizer

from src.DataGenerator import DataGenerator
from src.models.Model import Model
from src.models.AdvancedModel import AdvancedModel
from src.models.Xception import Xception
from src.Pipeline import Pipeline


Using TensorFlow backend.


In [2]:
# usefull setting variables
image_shape = 72
batch_size = 32
epochs = 3
data_train_path = 'data/train'
data_validation_path = 'data/validation'
data_test_path = 'data/test'
n_train_samples = 31367
n_valid_samples = 7842
n_test_samples = 12630
workers = 1
use_multiprocessing = False
pipeline = Pipeline()
pipeline.add_preprocessors((
            HistogramEqualizer(),
            Normalizer()
        ))

In [3]:
models = [
    {
        "title": "Simple Model Grayscale",
        "class": Model,
        "color_mode": "grayscale",
        "model_path": "model/simple_model-grayscale.json",
        "weights_path": "model/weights/simple_model-grayscale.h5",
        "history_path": "log/history_simple_model-grayscale.csv",
        "data_train_path": data_train_path,
        "data_validation_path": data_validation_path,
        "data_test_path": data_test_path,
        "model": None,
        "history": None,
        "scores": None,
    },
    {
        "title": "Advanced Model Grayscale",
        "class": AdvancedModel,
        "color_mode": "grayscale",
        "model_path": "model/advanced_model-grayscale.json",
        "weights_path": "model/weights/advanced_model-grayscale.h5",
        'history_path': 'log/history_advanced_model-grayscale.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Xception Model Grayscale',
        'class': Xception,
        'color_mode': 'grayscale',
        'model_path': 'model/xception_model-grayscale.json',
        'weights_path': 'model/weights/xception_model-grayscale.h5',
        'history_path': 'log/history_xception_model-grayscale.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Simple Model RGB',
        'class': Model,
        'color_mode': 'rgb',
        'model_path': 'model/simple_model-rgb.json',
        'weights_path': 'model/weights/simple_model-rgb.h5',
        'history_path': 'log/history_simple_model-rgb.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Advanced Model RGB',
        'class': AdvancedModel,
        'color_mode': 'rgb',
        'model_path': 'model/advanced_model-rgb.json',
        'weights_path': 'model/weights/advanced_model-rgb.h5',
        'history_path': 'log/history_advanced_model-rgb.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Xception Model RGB',
        'class': Xception,
        'color_mode': 'rgb',
        'model_path': 'model/xception_model-rgb.json',
        'weights_path': 'model/weights/xception_model-rgb.h5',
        'history_path': 'log/history_xception_model-rgb.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Simple Model Grayscale Using ROI',
        'class': Model,
        'color_mode': 'grayscale',
        'model_path': 'model/simple_model-grayscale.json',
        'weights_path': 'model/weights/simple_model-grayscale-roi.h5',
        'history_path': 'log/history_simple_model-grayscale-roi.csv',
        'data_train_path': data_train_path + '_roi',
        'data_validation_path': data_validation_path + '_roi',
        'data_test_path': data_test_path + '_roi',
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Advanced Model Grayscale Using ROI',
        'class': AdvancedModel,
        'color_mode': 'grayscale',
        'model_path': 'model/advanced_model-grayscale.json',
        'weights_path': 'model/weights/advanced_model-grayscale-roi.h5',
        'history_path': 'log/history_advanced_model-grayscale-roi.csv',
        'data_train_path': data_train_path + '_roi',
        'data_validation_path': data_validation_path + '_roi',
        'data_test_path': data_test_path + '_roi',
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Xception Model Grayscale Using ROI',
        'class': Xception,
        'color_mode': 'grayscale',
        'model_path': 'model/xception_model-grayscale.json',
        'weights_path': 'model/weights/xception_model-grayscale-roi.h5',
        'history_path': 'log/history_xception_model-grayscale-roi.csv',
        'data_train_path': data_train_path + '_roi',
        'data_validation_path': data_validation_path + '_roi',
        'data_test_path': data_test_path + '_roi',
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Simple Model RGB Using ROI',
        'class': Model,
        'color_mode': 'rgb',
        'model_path': 'model/simple_model-rgb.json',
        'weights_path': 'model/weights/simple_model-rgb-roi.h5',
        'history_path': 'log/history_simple_model-rgb-roi.csv',
        'data_train_path': data_train_path + '_roi',
        'data_validation_path': data_validation_path + '_roi',
        'data_test_path': data_test_path + '_roi',
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Advanced Model RGB Using ROI',
        'class': AdvancedModel,
        'color_mode': 'rgb',
        'model_path': 'model/advanced_model-rgb.json',
        'weights_path': 'model/weights/advanced_model-rgb-roi.h5',
        'history_path': 'log/history_advanced_model-rgb-roi.csv',
        'data_train_path': data_train_path + '_roi',
        'data_validation_path': data_validation_path + '_roi',
        'data_test_path': data_test_path + '_roi',
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Xception Model RGB Using ROI',
        'class': Xception,
        'color_mode': 'rgb',
        'model_path': 'model/xception_model-rgb.json',
        'weights_path': 'model/weights/xception_model-rgb-roi.h5',
        'history_path': 'log/history_xception_model-rgb-roi.csv',
        'data_train_path': data_train_path + '_roi',
        'data_validation_path': data_validation_path + '_roi',
        'data_test_path': data_test_path + '_roi',
        'model': None,
        'history': None,
        'scores': None,
    },
    
    
    
    {
        "title": "Simple Model Grayscale Augmented",
        "class": Model,
        "color_mode": "grayscale",
        "model_path": "model/simple_model-grayscale.json",
        "weights_path": "model/weights/simple_model-grayscale-augmented.h5",
        "history_path": "log/history_simple_model-grayscale-augmented.csv",
        "data_train_path": data_train_path,
        "data_validation_path": data_validation_path,
        "data_test_path": data_test_path,
        "model": None,
        "history": None,
        "scores": None,
    },
    {
        "title": "Advanced Model Grayscale Augmented",
        "class": AdvancedModel,
        "color_mode": "grayscale",
        "model_path": "model/advanced_model-grayscale.json",
        "weights_path": "model/weights/advanced_model-grayscale-augmented.h5",
        'history_path': 'log/history_advanced_model-grayscale-augmented.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Xception Model Grayscale Augmented',
        'class': Xception,
        'color_mode': 'grayscale',
        'model_path': 'model/xception_model-grayscale.json',
        'weights_path': 'model/weights/xception_model-grayscale-augmented.h5',
        'history_path': 'log/history_xception_model-grayscale-augmented.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Simple Model RGB Augmented',
        'class': Model,
        'color_mode': 'rgb',
        'model_path': 'model/simple_model-rgb.json',
        'weights_path': 'model/weights/simple_model-rgb-augmented.h5',
        'history_path': 'log/history_simple_model-rgb-augmented.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Advanced Model RGB Augmented',
        'class': AdvancedModel,
        'color_mode': 'rgb',
        'model_path': 'model/advanced_model-rgb.json',
        'weights_path': 'model/weights/advanced_model-rgb-augmented.h5',
        'history_path': 'log/history_advanced_model-rgb-augmented.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
    {
        'title': 'Xception Model RGB Augmented',
        'class': Xception,
        'color_mode': 'rgb',
        'model_path': 'model/xception_model-rgb.json',
        'weights_path': 'model/weights/xception_model-rgb-augmented.h5',
        'history_path': 'log/history_xception_model-rgb-augmented.csv',
        'data_train_path': data_train_path,
        'data_validation_path': data_validation_path,
        'data_test_path': data_test_path,
        'model': None,
        'history': None,
        'scores': None,
    },
]

### Test models

In [4]:
def create_model(model_dict):
    if model_dict['color_mode'] == 'rgb':
        shape = (image_shape, image_shape, 3)
    else:
        shape = (image_shape, image_shape, 1)
    model = model_dict['class']()
    
    model.load_model(model_dict['model_path'])
    model.compile()
    model.load_weights(model_dict['weights_path'])
    model.init_callbacks()
    
    model_dict['model'] = model

def read_history(model_dict):
    history = pd.read_csv(model_dict['history_path'], sep=",", header=0)
    model_dict['history'] = history
                             

def test_model(model_dict):
    test_generator = DataGenerator(
        model_dict['data_test_path'], 
        batch_size=batch_size,
        image_shape=(image_shape, image_shape),
        preprocessing_function=pipeline.evaluate
    )
    scores = model_dict['model'].evaluate_generator(
        test_generator.get_generator(color_mode=model_dict['color_mode']), 
        n_test_samples // batch_size)
    
    model_dict['scores'] = scores

In [8]:
scores_dataframe = pd.DataFrame(columns=['model', 'test_loss', 'test_acc'])
simple_model_scores = pd.DataFrame(columns=['model', 'test_loss', 'test_acc'])
advanced_model_scores = pd.DataFrame(columns=['model', 'test_loss', 'test_acc'])
xception_model_scores = pd.DataFrame(columns=['model', 'test_loss', 'test_acc'])
calculate_scores = False

i = 0
for model_dict in models:
    if model_dict['model'] is None:
        create_model(model_dict)
    if model_dict['history'] is None:
        read_history(model_dict)
    if model_dict['scores'] is None and calculate_scores:
        test_model(model_dict)
        scores_dataframe.loc[i] = [model_dict['title'], model_dict['scores'][0], model_dict['scores'][1]]
        if 'Simple Model' in model_dict['title']:
            simple_model_scores.loc[i] = [model_dict['title'], model_dict['scores'][0], model_dict['scores'][1]]
        if 'Advanced Model' in model_dict['title']:
            advanced_model_scores.loc[i] = [model_dict['title'], model_dict['scores'][0], model_dict['scores'][1]]
        if 'Xception Model' in model_dict['title']:
            xception_model_scores.loc[i] = [model_dict['title'], model_dict['scores'][0], model_dict['scores'][1]]
        i += 1
        scores_dataframe.set_index('model', inplace=True)
if calculate_scores is False:
    scores_dataframe = pd.read_csv('stats/cnn_test.csv', index_col=0)
    i, j, k = 0, 0, 0
    for index, row in scores_dataframe.iterrows():
        if 'Simple Model' in index:
            simple_model_scores.loc[i] = [index, row['test_loss'], row['test_acc']]
            i += 1
        if 'Advanced Model' in index:
            advanced_model_scores.loc[j] = [index, row['test_loss'], row['test_acc']]
            j += 1
        if 'Xception Model' in index:
            xception_model_scores.loc[k] = [index, row['test_loss'], row['test_acc']]
            k += 1

simple_model_scores.set_index('model', inplace=True)
advanced_model_scores.set_index('model', inplace=True)
xception_model_scores.set_index('model', inplace=True)

## Show Results

### Test Results

##### Simple Model

In [16]:
simple_model_scores.sort_values(by=['test_acc'], inplace=True, ascending=False)

simple_model_scores

,test_loss,test_acc
model,,
Simple Model RGB Augmented,0.201892,0.960660
Simple Model Grayscale,0.188426,0.960184
Simple Model RGB,0.209742,0.960184
Simple Model Grayscale Augmented,0.188796,0.958756
Simple Model RGB Using ROI,0.280281,0.949794
Simple Model Grayscale Using ROI,0.332303,0.943528


##### Advanced Model

In [10]:
advanced_model_scores.sort_values(by=['test_acc'], inplace=True, ascending=False)

advanced_model_scores

,test_loss,test_acc
model,,
Advanced Model RGB Augmented,0.093892,0.980171
Advanced Model RGB,0.111208,0.978585
Advanced Model Grayscale Augmented,0.097382,0.978426
Advanced Model Grayscale,0.111480,0.976919
Advanced Model Grayscale Using ROI,0.160788,0.970654
Advanced Model RGB Using ROI,0.171445,0.967005


##### Xception Model

In [11]:
xception_model_scores.sort_values(by=['test_acc'], inplace=True, ascending=False)

xception_model_scores

,test_loss,test_acc
model,,
Xception Model RGB,0.073178,0.981996
Xception Model RGB Augmented,0.078653,0.979933
Xception Model Grayscale,0.087862,0.977237
Xception Model Grayscale Augmented,0.096448,0.977237
Xception Model RGB Using ROI,0.104260,0.972557
Xception Model Grayscale Using ROI,0.138197,0.972081


##### All Models

In [12]:
scores_dataframe.sort_values(by=['test_acc'], inplace=True, ascending=False)
    
scores_dataframe

,test_loss,test_acc
model,,
Xception Model RGB,0.073178,0.981996
Advanced Model RGB Augmented,0.093892,0.980171
Xception Model RGB Augmented,0.078653,0.979933
Advanced Model RGB,0.111208,0.978585
Advanced Model Grayscale Augmented,0.097382,0.978426
Xception Model Grayscale,0.087862,0.977237
Xception Model Grayscale Augmented,0.096448,0.977237
Advanced Model Grayscale,0.111480,0.976919
Xception Model RGB Using ROI,0.104260,0.972557


### Models Summary 

##### Simple Model

In [13]:
models[0]['model'].model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 72, 72, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       73856     
__________

##### Advanced Model

In [14]:
models[1]['model'].model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 72, 72, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 70, 70, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 35, 35, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 33, 33, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
__________

##### Xception Model

In [15]:
models[2]['model'].model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 72, 72, 1)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con